In [ ]:
import distributed

client = distributed.Client()
client

In [ ]:
import fsspec
import xarray as xr
import xdggs

In [ ]:
url = "https://osi-saf.ifremer.fr/sst/l2p/global/avhrr_metop_b/2022/131/20220511231303-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20220511_231303-v02.0-fv01.0.nc"

In [ ]:
fs = fsspec.filesystem("http")
ds = xr.open_dataset(
    fs.open(url), engine="h5netcdf", chunks={}, decode_timedelta=True
)  # .pipe(lambda ds: ds.merge(ds.cf[["longitude", "latitude"]].compute()))
ds

In [ ]:
grid_info = xdggs.HealpixInfo(level=9, indexing_scheme="nested")

In [ ]:
def aggregation_regridding(grid_info, ds):
    lon_name = ds.cf.coordinates["longitude"][0]
    lat_name = ds.cf.coordinates["latitude"][0]

    stack_dims = list(dict.fromkeys(list(ds[lon_name].dims) + list(ds[lat_name].dims)))

    stacked = (
        ds.stack(points=stack_dims)
        .drop_indexes(["points"] + stack_dims)
        .drop_vars(stack_dims + ["points"])
    )
    cell_ids = xr.apply_ufunc(
        grid_info.geographic2cell_ids,
        stacked[lon_name].astype("float64"),
        stacked[lat_name].astype("float64"),
        keep_attrs=False,
        dask="parallelized",
    )

    return (
        stacked.assign_coords({"cell_ids": cell_ids})
        .groupby("cell_ids")
        .mean()
        .rename_dims({"cell_ids": "cells"})
        .dggs.decode(grid_info)
    )

In [ ]:
regridded = aggregation_regridding(grid_info, ds)
regridded

In [ ]:
computed = regridded.compute()
computed

In [ ]:
computed["sea_surface_temperature"].dggs.explore()